In [2]:
import pandas as pd

In [3]:
# Load count matrix
loaded_df = pd.read_excel('/Users/victoriagao/local_docs/NEST/stored_variables/count_matrix.xlsx')

In [4]:
loaded_df

,LGALS3-NPTN,FN1-RPSA,PTPRF-RACK1,INS-HLA,TIMP1-LRP1,THBS2-LRP1,THBS1-SDC1,MIF-CD74,LGALS3-ITGB4,RPS19-RPSA,...,COL18A1-ITGB5,HSPG2-DAG1,PSEN1-DAG1,TGFB2-ACVR1B,ITGB1-DAG1,BMP4-BMPR2,BMP2-ENG,HMGB1-AGTRAP,TGFB1-SDC2,no-ligand-no-receptor
0,2,0,2,0,0,0,1,0,12,1,...,0,0,1,0,1,1,0,0,0,1
1,1,0,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
2,6,55,27,2,21,2,18,9,21,26,...,0,0,0,1,0,0,1,0,0,0
3,2,0,0,3,0,0,2,0,23,0,...,0,4,0,0,0,0,0,0,0,0
4,2,0,7,0,2,0,0,1,3,1,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,9,1,...,0,0,1,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Putting everything together: Calculate Region VS Region for every one of 144 LR pair, put the results into a dataframe

In [5]:
# Regions of interset remember index starts from 0
interested_region = 0
comparing_region = 4

In [6]:
import pandas as pd

result_data = []

epsilon = 1e-10  # A very small value

for lr_pair in LR_pairs:
    
    # Create a DataFrame to store the contingency table
    columns = ['Region 1', 'Region 2']
    index = ['# of LR pair', '# of non LR pair']
    contingency_table = pd.DataFrame(index=index, columns=columns)
    contingency_table.fillna(0, inplace=True)  # Fill NaN cells with 0
    
    # # Calculate the sum of LR pairs other than lr_pair
    count_matrix_interested_row = count_matrix.iloc[interested_region]
    count_matrix_comparing_row = count_matrix.iloc[comparing_region]
    
    # # Populate the contingency table
    contingency_table.loc['# of LR pair','Region 1'] = count_matrix_interested_row[lr_pair]
    contingency_table.loc['# of non LR pair','Region 1'] = count_matrix_interested_row.drop(lr_pair).sum()
    
    contingency_table.loc['# of LR pair','Region 2'] = count_matrix_comparing_row[lr_pair]
    contingency_table.loc['# of non LR pair','Region 2'] = count_matrix_comparing_row.drop(lr_pair).sum()

    # Extract the relevant counts from the contingency table
    Ad = contingency_table.loc['# of LR pair','Region 1']
    Bc = contingency_table.loc['# of LR pair','Region 2']
    Bd = contingency_table.loc['# of non LR pair','Region 1']
    Ac = contingency_table.loc['# of non LR pair','Region 2']

    # Handle scenarios where Ad or Bc are 0
    if Ad == 0 and Bc != 0:
        log_odds_ratio = float('-inf')  # Negative infinity
    elif Bc == 0 and Ad != 0:
        log_odds_ratio = float('inf')   # Positive infinity
    else:
        # odds_ratio = (Ad / Bc) / (Bd / Ac)
        # log_odds_ratio = np.log(odds_ratio)
        Bc = max(Bc, epsilon)
        Bd = max(Bd, epsilon)
        Ac = max(Ac, epsilon)
        odds_ratio = (Ad / Bc) / (Bd / Ac)
    
        if odds_ratio == 0:  # To handle the case where odds_ratio is very close to 0
            log_odds_ratio = float('-inf')
        else:
            log_odds_ratio = np.log(odds_ratio)
        
    # Store the result in the result_data list
    result_data.append({'L-R Pair': lr_pair, 'log_odds_ratio': log_odds_ratio})

# Create a DataFrame from the result_data list
log_odds_df = pd.DataFrame(result_data)

log_odds_df = log_odds_df.sort_values(by='log_odds_ratio', ascending=True)

NameError: name 'LR_pairs' is not defined

In [60]:
log_odds_df.head(10)

,L-R Pair,log_odds_ratio
73,TGFB1-ACVR1,-inf
83,COL1A1-ITGA3,-inf
82,ADAM9-ITGA6,-inf
80,ITGB1-ITGA6,-inf
79,APP-SDC1,-inf
78,CDH1-ERBB2,-inf
77,CDH1-DDR1,-inf
76,TGFB1-ERBB2,-inf
125,ARF1-LRP5,-inf
74,HSP90B1-ERBB2,-inf
